In [1]:
import os
dirs = os.listdir(".")
print(dirs)

['Untitled.ipynb', 'api_search_sept17.py', 'current_directory.log', '__init__.py', 'unique_data.json', 'info.text-back', 'info.text', 'templates', 'current_directory.log.1', 'app.py', 'info.json', 'parse_info.py', 'api_search.py', '__pycache__', 'unique_data_sept17.json', '.ipynb_checkpoints']


In [7]:
with open("unique_data.json", "r", encoding="unicode_escape") as text_file:  
        file_content = text_file.read()
        #file_content = file_content.replace("URL:","<a href=")
        file_content =file_content.split("\n")
        for line in file_content:
            print(line)

[
    [
        {
            "role": "system",
            "content": "Here are some updated web searches. Use this to improve user response:[1] "This will open a window containing the HTML source code of the page. Inspect an HTML Element: Right-click on an element (or a blank area), and choose "Inspect" or "Inspect Element" to see what elements are made up of (you will see both the HTML and the CSS). You can also edit the HTML or CSS on-the-fly in the Elements or Styles panel that opens." URL:https://www.w3schools.com/html/html_basic.asp.[2] "HTML ( H yper T ext M arkup L anguage) is the code that is used to structure a web page and its content. For example, content could be structured within a set of paragraphs, a list of bulleted points, or using images and data tables. As the title suggests, this article will give you a basic understanding of HTML and its functions. So what is HTML?" URL:https://developer.mozilla.org/en-US/docs/Learn/Getting_started_with_the_web/HTML_basics.[3] "H

In [14]:
import re
input_text =[]
with open("unique_data.json", "r", encoding="unicode_escape") as text_file:  
        file_content = text_file.read()
        #file_content = file_content.replace("URL:","<a href=")
        file_content =file_content.split("\n")
        for line in file_content:
            input_text.append(line)


# Define a regular expression pattern to find URLs
url_pattern = r'URL:(https?://[^\s]+)'

# Function to convert URLs to hyperlinks
def convert_to_links(text):
    return re.sub(url_pattern, r'<a href="\1">\1</a>', text)

# Process the input_text and convert URLs
for item in input_text:
    item['content'] = convert_to_links(item['content'])
    print(item['content'])

# Print the modified input_text
#for item in input_text:
#    print(item['content'])


TypeError: string indices must be integers

In [ ]:
https://www.w3schools.com/html/html_basic.asp

In [6]:
import pprint
import json
import shutil
pp = pprint.PrettyPrinter(indent=4)
# Open the text file for reading
#with open("info.text", "r") as text_file:
with open("info.text", "r", encoding="unicode_escape") as text_file:    
    # Read the content of the file
    file_content = text_file.read()

    # Replace "][" with "],[" to separate the JSON objects with commas
    file_content = file_content.replace("][", "],[")

    # Wrap the content with square brackets to make it a valid JSON array
    json_content = f"[{file_content}]"

    # Parse the JSON data
    data = json.loads(json_content)
    #data = json_content

# Now 'data' contains a list of JSON objects
pp.pprint(data)
#print (data)

JSONDecodeError: Expecting ',' delimiter: line 9 column 100 (char 152)

In [ ]:
import pprint
import json
import shutil
pp = pprint.PrettyPrinter(indent=4)
# Open the text file for reading
with open("info.text", "r") as text_file:
    # Read the content of the file
    file_content = text_file.read()

    # Replace "][" with "],[" to separate the JSON objects with commas
    file_content = file_content.replace("][", "],[")

    # Wrap the content with square brackets to make it a valid JSON array
    json_content = f"[{file_content}]"

    # Parse the JSON data
    data = json.loads(json_content)

# Now 'data' contains a list of JSON objects
pp.pprint(data)

# Function to check for duplicate JSON objects
def is_duplicate(data, new_object):
    new_hash = hashlib.sha256(json.dumps(new_object, sort_keys=True).encode()).hexdigest()
    for existing_object in data:
        existing_hash = hashlib.sha256(json.dumps(existing_object, sort_keys=True).encode()).hexdigest()
        if new_hash == existing_hash:
            return True
    return False

# Define a list to store unique JSON objects

def json_convert():
    unique_data = []
    # Open the text file for reading
    with open("info.text", "r") as text_file:
        # Read the content of the file
        file_content = text_file.read()

        # Replace "][" with "],[" to separate the JSON objects with commas
        file_content = file_content.replace("][", "],[")

        # Wrap the content with square brackets to make it a valid JSON array
        json_content = f"[{file_content}]"

        # Parse the JSON data as a list of objects
        data = json.loads(json_content)

        # Iterate through the data and add unique objects to unique_data
        for obj in data:

            unique_data.append(obj)

        # Save the unique JSON data to a JSON file
        with open("unique_data.json", "w") as json_file:
            json.dump(unique_data, json_file, indent=4)
    shutil.copyfile('unique_data.json', 'templates/unique_data.html')        
    return unique_data


In [ ]:
from flask import request, Response
from requests import get
import json
def save_messages(info):
        """
        Save the messages to a file.
        :param messages: List of messages
        :param conversation_id: Conversation ID
        """
        filename = "info.json"
        with open(filename, 'a') as file:
            json.dump(info, file)
        return print("Info Saved")  
def fetch_search_results(query):
    """  
    Fetch search results for a given query.  

    :param query: Search query string  
    :return: List of search results  
    """
    search = get('https://ddg-api.herokuapp.com/search',
                 params={
                     'query': query,
                     'limit': 3,
                 })

    snippets = ""
    for index, result in enumerate(search.json()):
        snippet = f'[{index + 1}] "{result["snippet"]}" URL:{result["link"]}.'
        snippets += snippet

    response = "Here are some updated web searches. Use this to improve user response:"
    response += snippets

    return [{'role': 'system', 'content': response}]      
#----------------- 
query= input("Query: ")
info = fetch_search_results(query) 
save_messages(info)
info